### Welcome to a small Hysplit analysis tutorial

In [ ]:
### Conda Environment
# This file may be used to create an environment using: hysplit.yml
# To install the environment go to the terminal and type:
# $ conda env create -f hysplit.yml
# To activate this environment, use:
# $ conda activate hysplit
# now you are in the activated environment and have all packages installed
# to read HYSPLIT output files and plot them.
# starting working on the script type:
# $ jupyter notebook hysplit_tutorial.ipynb

# To deactivate an active environment, use:
# $ conda deactivate

In [1]:
# import all packages
import numpy as np
#import xarray as xr
#from netCDF4 import Dataset
import pandas as pd
import glob, os
from datetime import datetime
#packages needed for plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
#packages needed for clustering
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score
today_date = datetime.now().strftime("%Y_%m_%d")

ModuleNotFoundError: No module named 'xarray'

In [ ]:
path_traj = '/where/are/my/Hysplit/files'
output_path= '/where/should/my/output/files/go'

### Finding all Hysplit dat files
all_files= glob.glob(path_traj + "*.dat")
all_files.sort()

#common structure for HYSPLIT dat files in columns
#header= ['YEAR','MON',	'DAY',	'HOUR',	'MIN',	'SEC',	'TrajHour',	'LAT',	'LON',	'HEIGHT',	'PRESSURE',	'THETA',	'AIR_TEMP',	'RAINFALL',	'MIXDEPTH',	'RELHUMID',	'SPCHYUMID',	'H2OMIXRA',	'TERR_MSL', 'SUN_FLUX']

In [ ]:
# Reading necessary HYSPLIT columns as list
list_lla=[]
list_xyz=[]
pd_traj=pd.DataFrame()
for f in all_files:
    table = pd.read_table(f)     # read all files into a table
    for i,s in enumerate(table.iloc[:,0]):
        if '10 PRESSURE' in s:
            p = i
            break
    t=pd.read_csv(f, skiprows=p+2, sep='\s+', usecols=[9,10,11], names=['Lat','Lon','Height'])
    x,y,z= lla_to_xyz(t['Lat'], t['Lon'], t['Height'])  #conversion to x,y,z for clustering
    df = pd.DataFrame(data={'x': x, 'y': y})
    list_lla.append(t)
    list_xyz.append(df)

In [ ]:
#adding new strings for clustering dataframe header
#strings just count from 1 ... n, where n is lenth of the trajectory in hours
#for example 10 days backwards are 241 hours
n = 241
strs_x = ['x{}'.format(x) for x in range(n)]
strs_y = ['y{}'.format(x) for x in range(n)]
strs = strs_x + strs_y

traj_list=[]
for i in range(len(l_xyz)):
    traj_list.append(np.array(l_xyz[i]).flatten())

#change your dates and timedelta here
df_traj=pd.DataFrame(traj_list,index=pd.date_range(start='2009-01-01', periods=len(l_xyz), freq='3H'))
df_traj.columns= strs

In [ ]:
#Check if there are NaN values in the dataframe
nan_count= df_traj.isna().any().count()
nan_count

#in case of NaN values you will have to drop the rows

In [ ]:
#df_t is the dataframe without any NaN values
df_T= df_traj.dropna()
len(df_T), len(traj_list), len(df_traj) #checking the lenght for comparisons

In [ ]:
#decide on how many clusters
cluster_number=6
#clustering
#check documentation for more information
kmeans = KMeans(n_clusters=cluster_number, random_state=0).fit(df_T)

In [ ]:
#Output on clustering related information
print('number of interations, inertia and number of features:')
kmeans.n_iter_, kmeans.inertia_, kmeans.n_features_in_

In [ ]:
#cluster centres are your central trajectory for each group
cluster_centres=np.array(kmeans.cluster_centers_)
#reshape into numpy array with number of clusters, dimension (lat,lon) and trajectory length
cen_traj=np.reshape(cluster_centres, (cluster_number,2,n))


In [ ]:
#these are still in x,y coordinates,
#you can convert them back to lat,lon with the following function
#lat,lon= xyz_to_lla(x,y,z)
#try it yourself

In [ ]:
#kmeans.labels_ are the labels for each trajectory
# you could plot these as a histogram
#to see how many trajectories are in each cluster
plt.title('Frequency of cluster')
plt.hist(kmeans.labels_, bins=cluster_number)
plt.savefig(str(cluster_number)+'cluster_'+str(date)+'.png')
plt.show()

#maybe change the x ticks to 1 to cluster_number, since Python starts counting at 0



In [ ]:
#you can also plot the trajectories in a map
#Plot the trajectory centroids on a map
fig = plt.figure(figsize=(8,8))
ax = plt.axes(projection=ccrs.NorthPolarStereo())

ax.coastlines(linewidth=0.5) # Add features to axes object
 
# Add contour lines
ax.plot(ctraj1['y'],ctraj1['x'], color="b",transform=ccrs.PlateCarree(),label='cluster 1')
ax.plot(ctraj2['y'],ctraj2['x'], color="r",transform=ccrs.PlateCarree(),label='cluster 2')
ax.plot(ctraj3['y'],ctraj3['x'], color="g",transform=ccrs.PlateCarree(),label='cluster 3')
ax.plot(ctraj4['y'],ctraj4['x'], color="k",transform=ccrs.PlateCarree(),label='cluster 4')
ax.plot(ctraj5['y'],ctraj5['x'], color="y",transform=ccrs.PlateCarree(),label='cluster 5')
ax.plot(ctraj6['y'],ctraj6['x'], color="purple",transform=ccrs.PlateCarree(),label='cluster 6')

ax.set_extent([-180, 180, 63, 90], ccrs.PlateCarree())
plt.legend(loc= 'upper left', fontsize=14)
plt.savefig('cluster_traj_no'+str(cluster_number)+'_'+str(date)+'.png')

In [ ]:
#For checking how many clusters you want you could use the Davies Bouldin Score
results = {}
for i in range(2,15):
    kmeans = KMeans(n_clusters=i, random_state=30)
    labels = kmeans.fit_predict(df_T)
    db_index = davies_bouldin_score(df_T, labels)
    results.update({i: db_index})

plt.plot(results.values())
plt.xlabel("Number of clusters")
plt.ylabel("Davies-Boulding Index")
plt.show()

In [ ]:
#Plotting the Freqnecy of visits per gridbox can be done in a lot of ways
#here you can see how it is done for an hexbin grid

NUMBER_OF_TRAJECTORIES = len(df_T) # Don't forget keep this up to date!
HEXBIN_GRID_SIZE = 150  #How many hexbins do you want?
CLAT, CLON = 78.906, 11.888  #where is your receptor site?

# Data and map projection
data_crs = ccrs.PlateCarree()
map_crs  = ccrs.LambertAzimuthalEqualArea(central_longitude=CLON, central_latitude=CLAT)

# Initialise figure
from locale import normalize

fig, ax = plt.subplots(1, 1, subplot_kw={'projection': map_crs}, figsize=(8, 6))
ax.coastlines(resolution='50m', linewidth=0.5)
ax.gridlines(alpha=0.5, linestyle='--')

extent = 4500000  #how much map is supposedd to be seen?
ax.set_extent([-extent, extent, -extent, extent], crs=map_crs)
#You can also change the extent individually or change to lat,lon

# Plot station marker
stat_x, stat_y = ax.projection.transform_point(CLON, CLAT, src_crs=data_crs)
ax.plot(stat_x, stat_y, '^', markersize=5, color='red', zorder=10)

# Define C as a list of indices
C = np.arange(x.size)
img = ax.hexbin(x, y, C=C, transform=data_crs, gridsize=HEXBIN_GRID_SIZE, reduce_C_function=frequency_of_visit, vmin=0, vmax=18, linewidth=0.2, mincnt=5)
clb = fig.colorbar(img, ax=ax)
clb.ax.set_title('%')

# Add the centroid of the trajectories on top of the hexbin grid
#ax.plot(ctraj1['y'],ctraj1['x'], color="deepskyblue",transform=ccrs.Geodetic(),label='cluster 1')    
#plt.legend(loc= 'lower left', fontsize=14, ncol=2)

plt.savefig('hexbin150_'+str(date)+'.png', bbox_inches='tight', dpi=500)
plt.show()

Functions

In [1]:
def lla_to_xyz(lat, lon, alt):
    #conversion from lat,lon,alt to cartesian coordinates
    a = 6378137 
    f = 1/298.257224
    e = np.sqrt(2*f-f**2)
         
    lat = np.radians(lat)
    lon = np.radians(lon)
        
    #The eccentricity of the figure of the earth is found from    
    C = 1/(np.sqrt( (np.cos(lat)**2 + (1-f)**2 * (np.sin(lat))**2) ))        
    S = (1-f)**2 * C 

    x = (a*C+alt)*np.cos(lat)*np.cos(lon)
    y = (a*C+alt)*np.cos(lat)*np.sin(lon)
    z = (a*S+alt)*np.sin(lat)

    return x, y, z

In [2]:
def xyz_to_lla(x,y,z):
    #conversion from cartesian coordinates to lat,lon,alt
    import math
    #constants:
    a = 6378137 
    f = 1/298.257224
    e = np.sqrt(2*f-f**2)
    
    b   = np.sqrt(a**2*(1-e**2));
    ep  = np.sqrt((a**2-b**2)/b**2);
    p   = np.sqrt(x**2+y**2);
    th  = math.atan2(a*z,b*p);
    lon = math.atan2(y,x);
    lat = math.atan2((z+ep**2*b*math.sin(th)**3),(p-e**2*a*math.cos(th)**3));
    N   = a/np.sqrt(1-e**2*math.sin(lat)**2);
    alt = p/math.cos(lat)-N;
    
    lon = math.degrees(lon)
    lat = math.degrees(lat)
        
    return lat, lon, alt

In [ ]:
def frequency_of_visit(C):
    # C contains the values per step-point which are within a given hexagon
    # Frequency of visit expects indices of the flattened step-points i.e. an array like [0, 1, ..., n-1, n]
    C = np.array(C)

    # Calculate the number of unique visits to a given grid box
    # This accounts for trajectories looping through the same grid box multiple times
    mask = np.insert((np.diff(C) - 1).astype(bool), 0, True)
    visits = C[mask].size

    return visits / NUMBER_OF_TRAJECTORIES * 100